<a href="https://colab.research.google.com/github/cedro3/others/blob/master/ArtLine_make_gif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ArtLine_make_gif**
**Create** **Amazing** **Line** **Art**.

In [ ]:
# copy github
!git clone https://github.com/vijishmadhavan/ArtLine.git ArtLine
%cd ArtLine/

# get libralies
!pip install -r colab_requirements.txt
!pip install -q youtube-dl

# **Runtime**

* Hardware Accelerator = GPU              
You have to click twice


In [ ]:
import fastai
from fastai.vision import *
from fastai.utils.mem import *
from fastai.vision import open_image, load_learner, image, torch
import numpy as np
import urllib.request
import PIL.Image
from io import BytesIO
import torchvision.transforms as T
from PIL import Image
import requests
from io import BytesIO
import fastai
from fastai.vision import *
from fastai.utils.mem import *
from fastai.vision import open_image, load_learner, image, torch
import numpy as np
import urllib.request
import PIL.Image
from io import BytesIO
import torchvision.transforms as T

class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
              ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): self.hooks.remove()

In [ ]:
MODEL_URL = "https://www.dropbox.com/s/p9lynpwygjmeed2/ArtLine_500.pkl?dl=1 "
urllib.request.urlretrieve(MODEL_URL, "ArtLine_500.pkl")
path = Path(".")
learn=load_learner(path, 'ArtLine_500.pkl')

In [ ]:
# check YouTubeVideo
from IPython.display import YouTubeVideo
YOUTUBE_ID ='m0u0uAhoxq4'
YouTubeVideo(YOUTUBE_ID)

In [ ]:
# download YouTubeVideo
!rm -rf youtube.mp4
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

In [ ]:
# edit YouTubeVideo
import os
os.makedirs('video', exist_ok=True)
!ffmpeg -i youtube.mp4 -filter:v 'crop=300:300:170:0' -ss 00:00:01 -t 00:00:02 -async 1 ./video/takikuri.mp4

In [ ]:
# video2frames
import os
import cv2

os.makedirs('images', exist_ok=True)
 
def video_2_frames(video_file='./video/takikuri.mp4', 
                   image_dir='./images/', 
                   image_file='img_%s.png'):
 
    # Initial setting
    i = 0
    interval = 3
    length = 30
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(i).zfill(6), frame)
           print('Save', image_dir+image_file % str(i).zfill(6))
        i += 1 
    cap.release()  
 
def main():
    video_2_frames()
    
if __name__ == '__main__':
    main()  


In [ ]:
# frames2ArtLines
import os
import torchvision.utils as vutils

os.makedirs('out', exist_ok=True)

input_path = './images' 
output_path = './out'
files = os.listdir(input_path)
files.sort()

temp =[add_metrics]

for file in files:
  print(file)
  if file == '.ipynb_checkpoints':
     continue
  img = PIL.Image.open(input_path+'/'+file).convert("RGB")
  img_t = T.ToTensor()(img)
  img_fast = Image(img_t)
  p,img_hr,b = learn.predict(img_fast)
  vutils.save_image(img_hr,output_path+'/'+file)
 

In [ ]:
# ArtLines2GIF
from PIL import Image
import glob
 
files = sorted(glob.glob('./out/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('./takikuri.gif', save_all=True, 
               append_images=images[1:], 
               duration=100, loop=0)

In [ ]:
# display GIF
from IPython.display import Image
Image('./takikuri.gif', format='png')